# In this tutorial the operator "mapdl::run" is used to generate an rst file and then, results are evaluated on this rst

## lauch the server

In [1]:
from ansys import dpf
dpf.start_local_server()

server started at --address 10.110.2.47 --port 50052


## use "mapdl::run"

In [2]:
import os

if 'AWP_UNIT_TEST_FILES' in os.environ:
    unit_test_files = os.environ['AWP_UNIT_TEST_FILES']
    

test_file_path = os.path.join(unit_test_files, 'DataProcessing', 'rst_operators','presol','ShellTri','ds.dat')
data_sources = dpf.DataSources(test_file_path)

In [3]:
mapld_run =dpf.Operator("mapdl::run")
#print(mapld_run)

In [4]:
mapld_run.inputs.data_sources(data_sources)
mapld_run.inputs.mapdl_exe_path(r'C:\\Program Files\\ANSYS Inc\\v212\\ansys\\bin\\winx64\\ansys2021R2.exe')
rst_data_sources = mapld_run.outputs.data_sources()

## read diplacement, stress and strain

In [5]:
model = dpf.Model(rst_data_sources)
time_freq_support =model.time_freq_support
#print(time_freq_support)

In [6]:
assert time_freq_support.n_sets==1

In [7]:
mesh =model.meshed_region
#print(mesh)

In [8]:
els =mesh.elements
el = mesh.get_element(elementid=1)

In [9]:
assert len(els.ids)==20

In [10]:
assert el.nodes_count()==3
assert el.get_nodes()[0].coordinates ==[0.02006079886, 0.03, 0.0]
assert el.get_nodes()[2].coordinates ==[0.0168688973, 0.01999996456, 0.0]

### stress nodal

In [11]:
s = model.operator("S")
s.inputs.requested_location("Nodal")
fcS = s.outputs.fields_container()

In [12]:
min_max= dpf.Operator("min_max")
min_max.inputs.field.connect(fcS[0])

In [13]:
min = min_max.outputs.field_min()
max =  min_max.outputs.field_max()

In [15]:
print(min.data)
print(max.data)

[-4072515.75, -1221754.75, 0.0, -139656.62885799454, -50449.75721486132, -73258.63245792943]
[4072515.75, 1221754.75, 0.0, 139656.62885799454, 57722.45703125, 334181.59375000006]


In [16]:
assert min.data ==[-4072515.75, -1221754.75, 0.0, -139656.62885799454, -50449.75721486132, -73258.63245792943]

assert max.data ==[4072515.75, 1221754.75, 0.0, 139656.62885799454, 57722.45703125, 334181.59375000006]

In [17]:
assert len(fcS[0].scoping.ids)==20

### stress elemental nodal

In [18]:
s = model.operator("S")
fcS = s.outputs.fields_container()
min_max= dpf.Operator("min_max")
min_max.inputs.field.connect(fcS[0])
min = min_max.outputs.field_min()
max =  min_max.outputs.field_max()

In [19]:
assert min.data ==[-4072515.75, -1221754.75, 0.0, -235844.71875000038, -133905.73258208387, -156673.58243716997]

assert max.data ==[4072515.75, 1221754.75, 0.0, 235844.71875000038, 57722.45703125, 334181.59375000006]

### strain

In [20]:
s = model.operator("EPEL")
fcS = s.outputs.fields_container()
min_max= dpf.Operator("min_max")
min_max.inputs.field.connect(fcS[0])
min = min_max.outputs.field_min()
max =  min_max.outputs.field_max()

In [21]:
assert min.data ==[-1.852994682849385e-05, -3.1440763450518716e-06, -7.94140578364022e-06, -3.0659814456157666e-06, -0.0, 0.0]
assert max.data ==[1.852994682849385e-05, 3.1440763450518716e-06, 7.94140578364022e-06, 3.0659814456157666e-06, -0.0, 0.0]

### displacement 

In [22]:
u = model.operator("U")
fcU = u.outputs.fields_container()

In [23]:
min_max_over_time= dpf.Operator("min_max_fc")
min_max_over_time.inputs.fields_container.connect(fcU)
min = min_max_over_time.outputs.field_min()
max =  min_max_over_time.outputs.field_max()

In [24]:
assert min.data ==[0.0, 0.0, -3.6387417975137725e-05]

assert max.data ==[0.0, 0.0, 0.0]

## delete the temp folder

In [25]:
path =os.path.join(rst_data_sources.result_files[0])
import shutil
shutil.rmtree(os.path.dirname(path))